支持向量机SVM是从线性可分情况下的最优分类面提出的。所谓最优分类，就是要求分类线不但能够将两类无错误的分开，而且两类之间的分类间隔最大。推广到高维空间，最优分类线就成为最优分类面。

考虑这样一个问题：在一个n维欧式空间内有红蓝两组点，怎么找到一个超平面将它们分隔开？为了方便大家理解，考虑最简单的情况，平面直角坐标系内有红蓝两组点，用一条直线将它们分隔开，如图所示：


这似乎是很容易的一件事，直线的斜率可以不同，斜率相同的情况下截距也可以不同，有很多种分法都可以把两组样本点分割开来。但是，哪一种分法是最好的呢？这里首先就需要定义怎样才能算最好。不妨称：正样本到直线的距离与负样本到直线距离之和最大就是最好。样本到直线的距离定义为样本中所有点到直线距离的最小值。事实上，如果同学们还记得高中解析几何的知识，就知道点到直线的距离等于过这一点作直线平行线进而求解两平行线之间的距离。

所以，目标函数也就是求这样一条直线：过正样本和负样本中各自离直线最近的两点作平行线，使两平行线之间距离最大。为了表示样本的正负性，不妨记正样本标签为+1，负样本标签为-1，可以得到直线的数学表达形式：

这个时候我们就发现了问题：不是所有的样本都可以用一条直线粗暴地分割开的。即使用直线去分割，一定会有一些样本点会被分错。如何让支持向量机具备容错性呢？这时，就需要借助软间隔的概念。你可以理解为，软间隔是一种正则化手段，它允许支持向量机有一个较小的误差率。加入一些松弛变量模型就变成了：

很显然，经过映射以后，样本点用一个平面就可以完全把它们分开。那么这样的一个映射函数本质上是一个升维操作，这个函数也叫核函数。核方法使得数据能够由低维映射向高维，让形如异或问题这样的线性不可分问题得到解决。常用的核函数包括表8.2中列出的几种。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from tqdm import tqdm, trange
data = np.loadtxt('linear.csv', delimiter=',')
print('数据集大小：', len(data))
x = data[:, :2]
y = data[:, 2]
# 数据集可视化
plt.figure()
plt.scatter(x[y == -1, 0], x[y == -1, 1], color='red', label='y=-1')
plt.scatter(x[y == 1, 0], x[y == 1, 1], color='blue', marker='x', label='y=1')
plt.xlabel(r'$x_1$')
plt.ylabel(r'$x_2$')
plt.legend()
plt.show()

In [ ]:
def SMO(x, y, ker, C, max_iter):

    '''

    SMO算法

    x，y：样本的值和类别

    ker：核函数，与线性回归中核函数的含义相同

    C：惩罚系数

    max_iter：最大迭代次数

    '''

    # 初始化参数
    m = x.shape[0]
    alpha = np.zeros(m)
    # 预先计算所有向量的两两内积，减少重复计算
    K = np.zeros((m, m))
    for i in range(m):
        for j in range(m):
            K[i, j] = ker(x[i], x[j])
    for l in trange(max_iter):
        # 开始迭代
        for i in range(m):
            # 有m个参数，每一轮迭代中依次更新
            # 固定参数alpha_i与另一个随机参数alpha_j，并且保证i与j不相等
            j = np.random.choice([l for l in range(m) if l != i])
            # 用-b/2a更新alpha_i的值
            eta = K[j, j] + K[i, i] - 2 * K[i, j] # 分母
            e_i = np.sum(y * alpha * K[:, i]) - y[i] # 分子
            e_j = np.sum(y * alpha * K[:, j]) - y[j]
            alpha_i = alpha[i] + y[i] * (e_j - e_i) / (eta + 1e-5) # 防止除以0
            zeta = alpha[i] * y[i] + alpha[j] * y[j]
            # 将alpha_i和对应的alpha_j保持在[0,C]区间
            # 0 <= (zeta - y_j * alpha_j) / y_i <= C
            if y[i] == y[j]:
                lower = max(0, zeta / y[i] - C)

                upper = min(C, zeta / y[i])

            else:

                lower = max(0, zeta / y[i])

                upper = min(C, zeta / y[i] + C)

            alpha_i = np.clip(alpha_i, lower, upper)

            alpha_j = (zeta - y[i] * alpha_i) / y[j]
            # 更新参数
            alpha[i], alpha[j] = alpha_i, alpha_j
    return alpha
# 设置超参数
C = 1e8 # 由于数据集完全线性可分，我们不引入松弛变量
max_iter = 1000
np.random.seed(0)
alpha = SMO(x, y, ker=np.inner, C=C, max_iter=max_iter)
# 用alpha计算w，b和支持向量
sup_idx = alpha > 1e-5 # 支持向量的系数不为零
print('支持向量个数：', np.sum(sup_idx))
w = np.sum((alpha[sup_idx] * y[sup_idx]).reshape(-1, 1) * x[sup_idx], axis=0)
wx = x @ w.reshape(-1, 1)
b = -0.5 * (np.max(wx[y == -1]) + np.min(wx[y == 1]))
print('参数：', w, b)
# 绘图
X = np.linspace(np.min(x[:, 0]), np.max(x[:, 0]), 100)
Y = -(w[0] * X + b) / (w[1] + 1e-5)
plt.figure()
plt.scatter(x[y == -1, 0], x[y == -1, 1], color='red', label='y=-1')
plt.scatter(x[y == 1, 0], x[y == 1, 1], marker='x', color='blue', label='y=1')
plt.plot(X, Y, color='black')
# 用圆圈标记出支持向量
plt.scatter(x[sup_idx, 0], x[sup_idx, 1], marker='o', color='none', 
    edgecolor='purple', s=150, label='support vectors')
plt.xlabel(r'$x_1$')
plt.ylabel(r'$x_2$')
plt.legend()
plt.show()

一个线性不可分的例子

In [ ]:
data = np.loadtxt('spiral.csv', delimiter=',')

print('数据集大小：', len(data))

x = data[:, :2]

y = data[:, 2]

# 数据集可视化

plt.figure()

plt.scatter(x[y == -1, 0], x[y == -1, 1], color='red', label='y=-1')

plt.scatter(x[y == 1, 0], x[y == 1, 1], marker='x', color='blue', label='y=1')

plt.xlabel(r'$x_1$')

plt.ylabel(r'$x_2$')

plt.legend()

plt.axis('square')

plt.show()
# 简单多项式核

def simple_poly_kernel(d):

    def k(x, y): 

        return np.inner(x, y) ** d

    return k

# RBF核

def rbf_kernel(sigma):

    def k(x, y):

        return np.exp(-np.inner(x - y, x - y) / (2.0 * sigma ** 2))

    return k

# 余弦相似度核

def cos_kernel(x, y):

    return np.inner(x, y) / np.linalg.norm(x, 2) / np.linalg.norm(y, 2)

# sigmoid核

def sigmoid_kernel(beta, c):

    def k(x, y):

        return np.tanh(beta * np.inner(x, y) + c)

    return k

#测试不同核函数下样本会被如何核化：

kernels = [

    simple_poly_kernel(3), 

    rbf_kernel(0.1), 

    cos_kernel, 

    sigmoid_kernel(1, -1)

]

ker_names = ['Poly(3)', 'RBF(0.1)', 'Cos', 'Sigmoid(1,-1)']

C = 1e8

max_iter = 500

# 绘图准备，构造网格

plt.figure()

fig, axs = plt.subplots(2, 2, figsize=(10, 10))

axs = axs.flatten()

cmap = ListedColormap(['coral', 'royalblue'])

# 开始求解 SVM

for i in range(len(kernels)):

    print('核函数：', ker_names[i])

    alpha = SMO(x, y, kernels[i], C=C, max_iter=max_iter)

    sup_idx = alpha > 1e-5 # 支持向量的系数不为零

    sup_x = x[sup_idx] # 支持向量

    sup_y = y[sup_idx]

    sup_alpha = alpha[sup_idx]

    # 用支持向量计算 w^T*x

    def wx(x_new):

        s = 0

        for xi, yi, ai in zip(sup_x, sup_y, sup_alpha):

            s += yi * ai * kernels[i](xi, x_new)

        return s

    # 计算b*

    neg = [wx(xi) for xi in sup_x[sup_y == -1]]

    pos = [wx(xi) for xi in sup_x[sup_y == 1]]

    b = -0.5 * (np.max(neg) + np.min(pos))

    # 构造网格并用 SVM 预测分类

    G = np.linspace(-1.5, 1.5, 100)

    G = np.meshgrid(G, G)

    X = np.array([G[0].flatten(), G[1].flatten()]).T # 转换为每行一个向量的形式

    Y = np.array([wx(xi) + b for xi in X])

    Y[Y < 0] = -1

    Y[Y >= 0] = 1

    Y = Y.reshape(G[0].shape)

    axs[i].contourf(G[0], G[1], Y, cmap=cmap, alpha=0.5)

    # 绘制原数据集的点

    axs[i].scatter(x[y == -1, 0], x[y == -1, 1], color='red', label='y=-1')

    axs[i].scatter(x[y == 1, 0], x[y == 1, 1], marker='x', color='blue', label='y=1')

    axs[i].set_title(ker_names[i])

    axs[i].set_xlabel(r'$x_1$')

    axs[i].set_ylabel(r'$x_2$')

    axs[i].legend()

plt.show()